In [43]:
import pandas as pd
import re
import numpy as np
import functions

In [44]:
cur = functions.connectToPs()

Connecting to Postgree...

<connection object at 0x000001889EEB2E18; dsn: 'user=oxbigdatabi password=xxx dbname=bigdatabi host=bigdatabi.ckq5duwtue6h.eu-central-1.rds.amazonaws.com', closed: 0>


# Tabla Detalle

In [13]:
prueba = functions.detalle(cur)

In [15]:
prueba.to_excel('./output/detalle_cartera_20200127.xlsx',encoding='CP1252',index=False)

# Base

In [45]:
df = functions.base(cur)

In [4]:
# df[df['cif'] == 'B02580587']

In [4]:
print('Filtas Cartera:', len(df))
print('Total ventas:', len(df[df['isSold']==1]))
print('Total cancelaciones:', len(df[df['isCancelled']==1]))
print('Total assets:', len(df[df['isAsset']==1]))
print('Total bajas:', len(df[df['isBaja']==1]))
print('Total assets activos:', len(df[df['isActive']==1]))

Filtas Cartera: 1260
Total ventas: 1260
Total cancelaciones: 43
Total assets: 1137
Total bajas: 218
Total assets activos: 920


In [6]:
df.head(2)

,segmento,assetid,cif,canal_venta,product_name,migrado,deactivation_date,cancellation_date,rfb__c,purchase_date__c,Family,Velocity,isNeba,isSold,isAsset,isBaja,isActive,isCancelled
0,3-NOTRIAL,02i0N000006M6bnQAC,B85204915,DHO,Assets,None,NaT,NaT,2019-02-12 00:00:01,2019-02-12 00:00:01,X MUSIC,,0,1,1,0,1,0
1,3-NOTRIAL,02i0N000006M6boQAC,B85204915,DHO,Assets,None,NaT,NaT,2019-02-12 00:00:01,2019-02-12 00:00:01,X MUSIC,,0,1,1,0,1,0


In [6]:
df.to_excel('./output/df.xlsx',encoding='CP1252',index=False)

# VENTAS

In [5]:
functions.ass_vta_fam_mes(df,0)

purchase_year  2018            2019                                          \
purchase_month    9 10  11  12    1   2   3   4   5    6    7   8    9   10   
Family                                                                        
X FIBER           0  0   4   8    7  10   2   8  13   17   17   4   20   17   
X MUSIC           2  3   2   3    4   7   2   0   6    0    0   0    5    0   
X PRIVACY         0  0   5   4    0  12  14   4  12   13    6   2    9    6   
X PROTECTION      0  0   0   4    2   5  25  13   4   29   20   5   20   28   
X SECURITY        0  0   0   2    2   0   0   0   5   13    5   0   53   28   
X UCOM            0  0  13  13   20  22  11  16  22   65   57  12   59   94   
All               2  3  24  34   35  56  54  41  62  137  105  23  166  173   

purchase_year            2020   All  
purchase_month   11   12    1        
Family                               
X FIBER           2    3    8   140  
X MUSIC           0    0    1    35  
X PRIVACY         0    2   11   100  
X PROTECTION     28    4    9   196  
X SECURITY        2    2    1   113  
X UCOM           78  120   74   676  
All             110  131  104  1260

In [6]:
functions.ass_vta_prod_mes(df)

purchase_year                   2018            2019                       \
purchase_month                     9 10  11  12    1   2   3   4   5    6   
Family       product_name                                                   
X FIBER      X Fiber 100 - neba    0  0   0   0    1   2   0   1   2    0   
             X Fiber 100 - own     0  0   0   4    3   2   0   4   3   12   
             X Fiber 200 - neba    0  0   1   1    0   3   1   2   2    4   
             X Fiber 200 - own     0  0   0   1    0   0   0   0   6    0   
             X Fiber 50 - neba     0  0   1   1    1   3   0   0   0    0   
             X Fiber 50 - own      0  0   2   1    2   0   1   1   0    1   
X MUSIC      Assets                2  3   2   3    4   7   2   0   6    0   
X PRIVACY    X Privacy 100         0  0   0   0    0   2   0   2   0    3   
             X Privacy 200         0  0   0   1    0   0   0   0   4    5   
             X Privacy 50          0  0   5   3    0  10  14   2   8    5   
X PROTECTION Assets                0  0   0   4    2   5  25  13   4   29   
X SECURITY   X Security 100        0  0   0   2    2   0   0   0   5   11   
             X Security 200        0  0   0   0    0   0   0   0   0    2   
X UCOM       Teamwork+PBX          0  0  13  13   20  22  11  16  22   65   
All                                2  3  24  34   35  56  54  41  62  137   

purchase_year                                                2020   All  
purchase_month                     7   8    9   10   11   12    1        
Family       product_name                                                
X FIBER      X Fiber 100 - neba    1   1    1   13    0    1    1    24  
             X Fiber 100 - own     1   0   15    4    0    0    0    48  
             X Fiber 200 - neba    5   0    2    0    0    0    3    24  
             X Fiber 200 - own     2   1    2    0    1    2    2    17  
             X Fiber 50 - neba     4   1    0    0    1    0    0    12  
             X Fiber 50 - own      4   1    0    0    0    0    2    15  
X MUSIC      Assets                0   0    5    0    0    0    1    35  
X PRIVACY    X Privacy 100         0   0    0    2    0    2    0    11  
             X Privacy 200         1   0    4    4    0    0   10    29  
             X Privacy 50          5   2    5    0    0    0    1    60  
X PROTECTION Assets               20   5   20   28   28    4    9   196  
X SECURITY   X Security 100        2   0   50   27    0    2    0   101  
             X Security 200        3   0    3    1    2    0    1    12  
X UCOM       Teamwork+PBX         57  12   59   94   78  120   74   676  
All                              105  23  166  173  110  131  104  1260

In [7]:
functions.cli_vta_fam_mes(df,0)

purchase_year  2018            2019                                          \
purchase_month    9 10  11  12    1   2   3   4   5   6   7   8   9  10  11   
canal_venta                                                                   
DHO               0  2   6   7    9  23  14   7  15  34  43  12  35  40  36   
IT integrator     0  0   1   4    3   1   1   8   3   5   4   1   7   2   2   
WEB               1  1   4   8    1   1   7   4   7   6   4   3   9   7   4   
All               1  3  11  19   13  25  22  19  25  45  51  16  51  49  42   

purchase_year      2020  All  
purchase_month  12    1       
canal_venta                   
DHO             34   29  346  
IT integrator    1    1   44  
WEB              3    4   74  
All             38   34  464

# Altas

In [8]:
df_altas = functions.base_altas(df)

In [9]:
df_altas.head(2)

,cif,canal_venta,assets_act,rfb,deactivation_date,assets_tot,isClient,isActive,assets_dea,isBaja
0,03817373V,WEB,0,2019-11-20 00:00:01,2019-11-20 15:29:08,2,1,0,2,1
1,07477239P,DHO,0,2018-10-25 09:38:28,2019-01-21 09:13:32,1,1,0,1,1


In [13]:
df_altas.to_excel('./detalle_altas.xlsx',encoding='CP1252',index=False)

In [10]:
functions.cli_altas_bajas_mes(df_altas,canal=0)

year     2018        2019                                             2020
month      10 11  12   1   2   3   4   5   6   7   8   9   10  11  12   1 
isClient    3  5  18   13  20  14  18  21  36  36  21  40  44  39  25   29
isBaja      0  0   1    3   0   2   6   8   5   4   5   8  10   7   7    6

In [16]:
altas, bajas = functions.ass_altas_bajas_fam_mes(df, nivel ='Fam',canal = 1)
display(bajas)

year                       2018       2019                                    \
month                        10 11 12    1  2  3   4   5   6   7   8   9  10   
canal_venta   Family                                                           
DHO           X FIBER         0  0  0    0  0  1   0   0   0   3   3   1   1   
              X MUSIC         0  0  0    1  0  1   1   0   0   0   0   0   0   
              X PRIVACY       0  0  0    0  0  0   4   4   2   2  10   2   5   
              X PROTECTION    0  0  0    0  0  0   0   2   3   0   1   1   1   
              X SECURITY      0  0  0    0  0  0   0   0   0   2   0   0   3   
              X UCOM          0  0  0    3  1  6   1   1   4  10   5   5   9   
IT integrator X FIBER         0  0  0    0  0  0   0   0   0   0   0   2   0   
              X MUSIC         0  0  0    0  0  0   0   1   0   0   0   1   0   
              X PRIVACY       0  0  0    0  0  0   0   0   0   0   0   0   0   
              X PROTECTION    0  0  0    0  0  0   1   0   1   0   0   1   0   
              X SECURITY      0  0  0    0  0  0   0   0   0   0   0   0   0   
              X UCOM          0  0  0    0  0  0   0   2   0   0   0   0   0   
WEB           X FIBER         0  0  0    0  0  1   0   0   1   0   0   0   1   
              X MUSIC         0  0  1    0  0  0   1   1   0   0   1   0   0   
              X PRIVACY       0  0  0    0  0  0   2   0   0   0   0   0   0   
              X PROTECTION    0  0  0    2  0  0   1   1   0   0   0   0   0   
              X UCOM          0  0  0    0  1  0   0   0   3   1   4  14   5   
All                           0  0  1    6  2  9  11  12  14  18  24  27  25   

year                               2020  All  
month                       11  12    1       
canal_venta   Family                          
DHO           X FIBER        0   1    1   11  
              X MUSIC        0   0    0    3  
              X PRIVACY      0   0    0   29  
              X PROTECTION   2   2    6   18  
              X SECURITY     0   3    0    8  
              X UCOM         4  11   20   80  
IT integrator X FIBER        0   1    0    3  
              X MUSIC        0   0    0    2  
              X PRIVACY      0   0    0    0  
              X PROTECTION   4   0    0    7  
              X SECURITY     0   0    0    0  
              X UCOM         0   0    0    2  
WEB           X FIBER        0   0    1    4  
              X MUSIC        0   0    0    4  
              X PRIVACY      0   0    0    2  
              X PROTECTION   7   0    0   11  
              X UCOM         3   2    0   33  
All                         20  20   28  217

## Migras

In [46]:
mig_df =  functions.migBase(cur)

In [47]:
mig_df.head(2)

,assetid,assethijo_mig,product_name,Family,cif,canal_venta,mig_date,tipo_migra,month_year
0,02i0N00000I3wptQAB,None,Teamwork+PBX,X UCOM,B98115520,DHO,2019-05-09 17:21:03,IN,05-19
1,02i0N00000KmJaoQAF,None,Teamwork+PBX,X UCOM,B78909611,DHO,2019-07-19 16:34:33,IN,07-19


In [48]:
check = list(set(mig_df.month_year))

In [60]:
for date in months: 
    if date not in check:
        print(date)
        mig_df = mig_df.append({'month_year':date},ignore_index=True)

09-18
10-18
11-18
12-18
01-19
02-19
03-19
12-19


'10-18'

In [63]:
mig_df.tail(10)

,assetid,assethijo_mig,product_name,Family,cif,canal_venta,mig_date,tipo_migra,month_year
42,02i0N00000Nv2yTQAR,02i0N00000NvL0HQAV,Teamwork+PBX,X UCOM,B82039637,DHO,2019-11-18 10:29:49,OUT,11-19
43,02i0N00000Pfg96QAB,02i0N00000PfMuWQAV,Teamwork+PBX,X UCOM,B76686716,DHO,2020-01-07 08:52:01,OUT,01-20
44,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,09-18
45,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,10-18
46,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,11-18
47,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,12-18
48,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,01-19
49,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,02-19
50,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,03-19
51,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,12-19


In [64]:
group = mig_df[['assetid']].groupby(by=[mig_df.tipo_migra,mig_df.mig_date.dt.year, mig_df.mig_date.dt.month,mig_df.Family,mig_df.product_name]).count()
group.index.names  = ['tipo_migra','year', 'month', 'Family', 'product_name']

group.reset_index(inplace=True)
migin = group[group['tipo_migra']=='IN']
# pivot_migin = migin.pivot_table(index='Family', columns=['year','month'], values='assetid')
migin

,tipo_migra,year,month,Family,product_name,assetid
0,IN,2019,4,X FIBER,X Fiber 100 - neba,1
1,IN,2019,5,X UCOM,Teamwork+PBX,11
2,IN,2019,6,X UCOM,Teamwork+PBX,1
3,IN,2019,7,X UCOM,Teamwork+PBX,2
4,IN,2019,8,X UCOM,Teamwork+PBX,1
5,IN,2019,9,X UCOM,Teamwork+PBX,1
6,IN,2019,10,X UCOM,Teamwork+PBX,3
7,IN,2019,11,X UCOM,Teamwork+PBX,1
8,IN,2020,1,X UCOM,Teamwork+PBX,1


In [67]:
import datetime

months = []
for date in pd.date_range(df.purchase_date__c.min(), df.purchase_date__c.max()+ datetime.timedelta(days=30), freq='M'):
    months.append(date.strftime('%m-%Y'))
    
months 

['09-2018',
 '10-2018',
 '11-2018',
 '12-2018',
 '01-2019',
 '02-2019',
 '03-2019',
 '04-2019',
 '05-2019',
 '06-2019',
 '07-2019',
 '08-2019',
 '09-2019',
 '10-2019',
 '11-2019',
 '12-2019',
 '01-2020']

In [24]:
df.purchase_date__c.max()+ datetime.timedelta(days=20)

Timestamp('2020-02-13 00:00:01')

In [64]:
df['mes_year']= df.purchase_date__c.apply(lambda x: x.strftime('%m-%Y'))

In [96]:
df

,assetid,product_name,Family,cif,canal_venta,purchase_date__c,cancellation_date,rfb__c,deactivation_date,mig_in_date,migrado,isSold,isCancelled,isAsset,isActive,isBaja,mes_year
0,02i0N000006M6bnQAC,Assets,X MUSIC,B85204915,DHO,2019-02-12 00:00:01,NaT,2019-02-12 00:00:01,NaT,NaT,None,1,0,1,1,0,02-19
1,02i0N000006M6boQAC,Assets,X MUSIC,B85204915,DHO,2019-02-12 00:00:01,NaT,2019-02-12 00:00:01,NaT,NaT,None,1,0,1,1,0,02-19
2,02i0N000006M6bpQAC,Assets,X MUSIC,B85204915,DHO,2019-02-12 00:00:01,NaT,2019-02-12 00:00:01,NaT,NaT,None,1,0,1,1,0,02-19
3,02i0N000006M6bqQAC,Assets,X MUSIC,B85204915,DHO,2019-02-12 00:00:01,NaT,2019-02-12 00:00:01,NaT,NaT,None,1,0,1,1,0,02-19
4,02i0N000006M9PwQAK,Assets,X MUSIC,B63351431,DHO,2019-02-13 00:00:01,NaT,2019-02-13 00:00:01,NaT,NaT,None,1,0,1,1,0,02-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,02i0N00000PgSpcQAF,Teamwork+PBX,X UCOM,B16523219,DHO,2020-01-24 00:00:01,NaT,2020-01-24 00:00:01,NaT,NaT,None,1,0,1,1,0,01-20
1256,02i0N00000PgSpdQAF,Teamwork+PBX,X UCOM,B16523219,DHO,2020-01-24 00:00:01,NaT,2020-01-24 00:00:01,NaT,NaT,None,1,0,1,1,0,01-20
1257,02i0N00000PgSytQAF,Teamwork+PBX,X UCOM,B25401738,DHO,2020-01-24 00:00:01,NaT,2020-01-24 00:00:01,NaT,NaT,None,1,0,1,1,0,01-20
1258,02i0N00000PgSyuQAF,Teamwork+PBX,X UCOM,B25401738,DHO,2020-01-24 00:00:01,NaT,2020-01-24 00:00:01,NaT,NaT,None,1,0,1,1,0,01-20
